In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [32]:
# path
data_path = "../moa_data/"
data_file_list = ["train_features.csv", "test_features.csv", "train_targets_scored.csv", \
                  "train_targets_nonscored.csv"]

# load data
data_train = pd.read_csv(data_path + data_file_list[0])
data_test = pd.read_csv(data_path + data_file_list[1])
train_targets_scored = pd.read_csv(data_path + data_file_list[2])
train_targets_nonscored = pd.read_csv(data_path + data_file_list[3])

# data info
print(f'Training features file: {data_train.shape[0]} rows; {data_train.shape[1]} columns')
print(f'Testing features file: {data_test.shape[0]} rows; {data_test.shape[1]} columns')
print(f'Training targets scored file: {train_targets_scored.shape[0]} rows; {train_targets_scored.shape[1]} columns')
print(f'Training targets nonscored file: {train_targets_nonscored.shape[0]} rows; {train_targets_nonscored.shape[1]} columns')

Training features file: 23814 rows; 876 columns
Testing features file: 3982 rows; 876 columns
Training targets scored file: 23814 rows; 207 columns
Training targets nonscored file: 23814 rows; 403 columns


In [13]:
data_train.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [14]:
data_test.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_0004d9e33,trt_cp,24,D1,-0.5458,0.1306,-0.5135,0.4408,1.5500,-0.1644,...,0.0981,0.7978,-0.1430,-0.2067,-0.2303,-0.1193,0.0210,-0.0502,0.1510,-0.7750
1,id_001897cda,trt_cp,72,D1,-0.1829,0.2320,1.2080,-0.4522,-0.3652,-0.3319,...,-0.1190,-0.1852,-1.0310,-1.3670,-0.3690,-0.5382,0.0359,-0.4764,-1.3810,-0.7300
2,id_002429b5b,ctl_vehicle,24,D1,0.1852,-0.1404,-0.3911,0.1310,-1.4380,0.2455,...,-0.2261,0.3370,-1.3840,0.8604,-1.9530,-1.0140,0.8662,1.0160,0.4924,-0.1942
3,id_00276f245,trt_cp,24,D2,0.4828,0.1955,0.3825,0.4244,-0.5855,-1.2020,...,0.1260,0.1570,-0.1784,-1.1200,-0.4325,-0.9005,0.8131,-0.1305,0.5645,-0.5809
4,id_0027f1083,trt_cp,48,D1,-0.3979,-1.2680,1.9130,0.2057,-0.5864,-0.0166,...,0.4965,0.7578,-0.1580,1.0510,0.5742,1.0900,-0.2962,-0.5313,0.9931,1.8380


In [16]:
train_targets_scored.sample(5)

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
902,id_0921fcf48,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7017,id_4b3977d61,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2759,id_1d9111c62,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20442,id_db7b09c90,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7892,id_549f82153,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Note: cp_type indicates samples treated with a compound (cp_vehicle) or with a control perturbation (ctrl_vehicle); **control perturbations have no MoAs**.

In [7]:
data_train.cp_type.value_counts(normalize=True)

trt_cp         0.921643
ctl_vehicle    0.078357
Name: cp_type, dtype: float64

In [23]:
control_group = data_train.loc[data_train.cp_type == 'ctl_vehicle']
control_group['sig_id'].count()

1866

### Pre-processing ordered data into dummy values

In [25]:
# expand 7 dummy features 
data_train_processed = pd.concat([data_train, pd.get_dummies(data_train['cp_time'], prefix='cp_time')], axis=1)
data_train_processed = pd.concat([data_train_processed, pd.get_dummies(data_train['cp_dose'], prefix='cp_dose')], axis=1)
data_train_processed = pd.concat([data_train_processed, pd.get_dummies(data_train['cp_type'], prefix='cp_type')], axis=1)

# drop the three old features
data_train_processed = data_train_processed.drop(['cp_type', 'cp_time', 'cp_dose'], axis=1)
data_train_processed.sample(5)

,sig_id,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,...,c-97,c-98,c-99,cp_time_24,cp_time_48,cp_time_72,cp_dose_D1,cp_dose_D2,cp_type_ctl_vehicle,cp_type_trt_cp
16271,id_af0e29cbd,-0.2491,-0.4834,0.6935,1.3480,-0.3309,0.2232,-0.2886,0.7153,-0.4083,...,-1.1930,-0.1993,0.2469,0,0,1,0,1,0,1
21212,id_e38b76d85,1.4910,-0.5825,-0.2730,-0.1919,0.0554,1.4320,0.8605,-3.0060,0.8875,...,0.6054,0.0758,-0.9174,0,0,1,0,1,0,1
13948,id_9643ea596,0.3958,0.2084,0.7189,-1.0260,2.1930,-3.6640,1.1050,1.2160,0.6289,...,-0.8395,-0.1374,-0.3115,0,0,1,0,1,0,1
21268,id_e41ea7fa5,0.3913,-0.1797,-0.6755,-0.9256,-0.2419,-3.7670,-0.0796,0.7967,0.4875,...,0.5444,-0.1222,-1.0500,0,0,1,0,1,0,1
15881,id_aacfa27a5,-0.3554,0.1982,-2.7860,1.5050,1.4450,-0.7909,0.1274,-1.6090,0.6917,...,0.4326,-3.9380,-0.8892,0,0,1,0,1,0,1


In [33]:
train_valid = data_train.iloc[:, 1:].values